In [ ]:
import os
os.chdir("/LOCAL/jjerome/GitHub/rl4mm/")

from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime, timedelta
import pandas as pd

from rl4mm.agents.baseline_agents import Teradactyl, FixedActionAgent
from rl4mm.database.HistoricalDatabase import HistoricalDatabase
from rl4mm.features.Features import Inventory
from rl4mm.gym.action_interpretation.OrderDistributors import BetaOrderDistributor
from rl4mm.gym.HistoricalOrderbookEnvironment import HistoricalOrderbookEnvironment
from rl4mm.gym.order_tracking.InfoCalculators import SimpleInfoCalculator
from rl4mm.gym.utils import generate_trajectory
from rl4mm.simulation.HistoricalOrderGenerator import HistoricalOrderGenerator
from rl4mm.simulation.OrderbookSimulator import OrderbookSimulator

plt.rcParams["figure.figsize"] = (10,6)
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

In [ ]:
TOTAL_VOLUME = 100
max_inventory = 250
enter_spread = False
step_size = timedelta(seconds = 5)
min_quote_level = 0
max_quote_level = 20

In [ ]:
a = 1
b = 2
default_omega = Teradactyl.calculate_omega(a, b)
default_kappa = Teradactyl.calculate_kappa(a, b)

In [ ]:
min_date = datetime(2022,3,2)
max_date = datetime(2022,3,2)
ticker = "JPM"
episode_length = timedelta(minutes=60)
timesteps = np.arange(0,60,1/12)
fixed = FixedActionAgent(np.array([1,2,1,2, max_inventory]))
min_start_timedelta=timedelta(hours = 10, minutes=30)
max_end_timedelta=timedelta(hours=11, minutes=30)

In [ ]:
def get_env(market_order_clearing:bool=False, market_order_fraction_of_inventory:float=0.0):
    simulator = OrderbookSimulator(
            ticker=ticker,
            order_generators=[HistoricalOrderGenerator(ticker, HistoricalDatabase())],
            n_levels=50,
            episode_length=episode_length,
        )
    order_distributor = BetaOrderDistributor(quote_levels=max_quote_level- min_quote_level, active_volume=TOTAL_VOLUME)
    info_calculator = SimpleInfoCalculator(market_order_fraction_of_inventory=market_order_fraction_of_inventory)
    return HistoricalOrderbookEnvironment(features=[Inventory()],
                                          min_date=min_date,
                                          simulator=simulator,
                                          max_date=max_date,
                                          min_start_timedelta=min_start_timedelta,
                                          max_end_timedelta=max_end_timedelta, 
                                          episode_length=episode_length,
                                          min_quote_level=min_quote_level,
                                          max_quote_level=max_quote_level,
                                          enter_spread=enter_spread,
                                          step_size = step_size,
                                          ticker= "JPM",
                                          market_order_clearing=market_order_clearing,
                                          market_order_fraction_of_inventory=market_order_fraction_of_inventory, 
                                          preload_orders=True,
                                          info_calculator=info_calculator)

def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

trajectories = {}
market_order_types = {"no_market":(False, 0.0), "full_market":(True, 1.0), "half_market":(True,0.5)}

## Market Order Clearing

In [ ]:
for name, value in market_order_types.items():
    env = get_env(value[0],value[1])
    trajectories[name] = generate_trajectory(fixed,env)

In [ ]:
fig, ax_left = plt.subplots()

ax_left.plot(timesteps,np.cumsum(trajectories["no_market"]["rewards"])/10000, label = "No MOC", color='r', linestyle = ":", linewidth = 1.5)
ax_left.plot(timesteps,np.cumsum(trajectories["full_market"]["rewards"])/10000, label = r"MOC with frac_inv = 1", color='k', linestyle="-", linewidth = 1.5)
ax_left.plot(timesteps,np.cumsum(trajectories["half_market"]["rewards"])/10000, label = r"MOC with frac_inv = 0.5", color='k', linestyle=":", linewidth = 1.5)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.ylabel(r'PnL (\$)', fontsize=20)
plt.xlabel(r'Time (mins)', fontsize=20)
ax_left.legend(loc="upper left", fontsize=15)
plt.title(r'PnL with and without market order clearing (MOC)', fontsize=25, y = 1.01)
plt.show()
# plt.savefig("/home/staffi/ecco/jjerome/Documents/fixed_and_market_order_rewards.pdf")

In [ ]:
fig, ax_left = plt.subplots()
ax_left.plot(timesteps,np.array([info['inventory'] for info in trajectories["no_market"]["infos"]]), label = "No MOC", color='r', linestyle = ":", linewidth = 1.5)
ax_left.plot(timesteps,np.array([info['inventory'] for info in trajectories["full_market"]["infos"]]), label = r"MOC with frac_inv = 1", color='black', linestyle="-", linewidth = 1.5)
ax_left.plot(timesteps,np.array([info['inventory'] for info in trajectories["half_market"]["infos"]]), label = r"MOC with frac_inv = 0.5", color='black', linestyle=":", linewidth = 1.5)
plt.ylabel(r'Inventory', fontsize=20)
ax_left.legend(loc="lower left", fontsize=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel(r'Time (mins)', fontsize=20)


plt.title(r'Agent inventory with and without market order clearing (MOC)', fontsize=25, y = 1.02)
# plt.savefig("/home/staffi/ecco/jjerome/Documents/fixed_and_market_order_inventory.pdf")
plt.show()

## Teradactyl (midprice skewing agent)

In [ ]:
default_kappa = 7
default_omega = 0.5
max_kappa = default_kappa*30
exponent = 2
max_inventory = 1000

In [ ]:
tera = Teradactyl(max_inventory=max_inventory,
                                default_kappa = default_kappa,
                                default_omega=default_omega,
                                max_kappa=max_kappa, 
                                exponent=exponent, 
                                inventory_index=0)

In [ ]:
env = get_env()

In [ ]:
tera_trajectory = generate_trajectory(tera,env)

In [ ]:
timesteps = np.arange(0,60,1/12)


In [ ]:
fig, ax_left = plt.subplots()
ax_left.plot(timesteps,np.array([info["asset_price"] for info in tera_trajectory["infos"]])/10000, label = "Asset price", color='r', linestyle = ":", linewidth = 1.5)

plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.ylabel(r'Asset price (\$)', fontsize=20)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel(r'Time (mins)', fontsize=20)

ax_right = ax_left.twinx()

ax_right.plot(timesteps,np.cumsum(tera_trajectory["rewards"])/10000, label = "PnL", color='k', linestyle="-", linewidth = 1.5)
ax_left.legend(loc="upper left", fontsize=15)
ax_right.legend(loc="lower left", fontsize=15)
plt.ylabel(r'PnL (\$)', fontsize=20)
ax_right.yaxis.set_label_coords(1.09, 0.5)

plt.title(r'Trajectory of the underlying asset price and agent PnL', fontsize=25, y = 1.01)

plt.yticks(fontsize=15)
plt.xticks(fontsize=15)

# plt.savefig("/home/staffi/ecco/jjerome/Documents/asset_price_and_rewards.pdf")
plt.show()

In [ ]:
fig, ax_left = plt.subplots()
ax_left.plot(timesteps,np.array([info['inventory'] for info in tera_trajectory["infos"]]), label = "Inventory", color='k', linestyle = "-")

plt.ylabel(r'Inventory', fontsize=20)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel(r'Time (mins)', fontsize=20)
ax_right = ax_left.twinx()

ax_right.plot(timesteps[9:],moving_average(np.array([info['weighted_midprice_offset'] for info in tera_trajectory["infos"]]),10), label = 'Midprice offset', color='r', linestyle=":")
ax_left.legend(loc="lower right", fontsize=15)
ax_right.legend(loc="upper right", fontsize=15)
plt.ylabel(r'Midprice offset', fontsize=20)

plt.title("Inventory and midprice offset", fontsize=25, y = 1.01)


plt.yticks(fontsize=15)
plt.xticks(fontsize=15)

# plt.savefig("/home/staffi/ecco/jjerome/Documents/inventory_midprice.pdf")

plt.show()

## Takeaways
- kappa needs to be quite high for the agent to successfully clear inventory.
- Perhaps kappa should increase in absolute inventory as well as omega
- should the default omega increase with step size?
- State dependent omega and kappa?